In [1]:
import tensorflow as tf
import math
from tensorflow.keras import optimizers
import numpy as np

In [3]:
class NaiveDense :
    def __init__(self, input_size, output_size, activation):
        self.activation = activation
        
        w_shape = (input_size, output_size) 
        w_initial_value = tf.random.uniform(w_shape, minval = 0, maxval=1e-1) # 랜덤한 값을 가진 가중치의 행렬 (input_size, output_size) W를 만듬
        self.W = tf.Variable(w_initial_value)
        
        b_shape = (output_size)
        b_initial_value = tf.zeros(b_shape) # 0으로 초기화된 (ouput_size,) 크기의 벡터 b를 생성
        self.b = tf.Variable(b_initial_value)

    def __call__(self, inputs) :
        return self.activation(tf.matmul(inputs, self.W) + self.b)
    
    @property
    def weights(self) : # 층의 가중치를 추출하기 위한 메서드
        return [self.W, self.b]

In [5]:
class NaiveSequential :
    def __init__(self, layers) :
        self.layers = layers
    
    def __call__(self, inputs) :
        x = inputs
        for layer in self.layers :
            x = layer(x)
        return x
    
    @property
    def weights(self) :
        weights = []
        for layer in self.layers :
            weights += layer.weights
        return weights 

In [15]:
model = NaiveSequential([
    NaiveDense(input_size=28 * 28, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size=512, output_size=10, activation=tf.nn.softmax)
])

assert len(model.weights) == 4

In [20]:
class BatchGenerator :
    def __init__(self, images, labels, batch_size=128) :
        assert len(images) == len(labels)
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.num_batches = math.ceil(len(images) / batch_size)
    
    def next(self) :
        images = self.images[self.index : self.index + self.batch_size]
        labels = self.labels[self.index : self.index + self.batch_size]
        self.index += self.batch_size
        return images, labels

In [17]:
def one_training_step(model, images_batch, labels_batch) :
    
    learning_rate = 1e-3
    optimizer = optimizers.SGD(learning_rate = 1e-3)
    
    def update_weights(gradients, weights) :
        #for g, w in zip(gradients, weights) :
        #    w.assign_sub(g * learning_rate)
        optimizer.apply_gradients(zip(gradients, weights))
            
    with tf.GradientTape() as tape :
        predictions = model(images_batch)
        per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(
            labels_batch, predictions)
        average_loss = tf.reduce_mean(per_sample_losses)
    gradients = tape.gradient(average_loss, model.weights)
    update_weights(gradients, model.weights)
    return average_loss



In [18]:
def fit(model, images, labels, epochs, batch_size=128) :
    for epoch_counter in range(epochs) :
        print(f"에포크 {epoch_counter} ")
        batch_generator = BatchGenerator(images, labels)
        for batch_counter in range(batch_generator.num_batches) :
            images_batch, labels_batch = batch_generator.next()
            loss = one_training_step(model, images_batch, labels_batch)
            if batch_counter % 100 == 0:
                print(f"{batch_counter}번째 배치 손실: {loss:.2f}")

In [21]:
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data() # MNIST 에 있는 훈련 세트와 테스트 세트를 가져와 저장


train_images = train_images.reshape((60000, 28 * 28))
# 60000, 784
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

fit(model, train_images, train_labels, epochs=10, batch_size=128)

에포크 0 
0번째 배치 손실: 3.44
100번째 배치 손실: 2.22
200번째 배치 손실: 2.18
300번째 배치 손실: 2.10
400번째 배치 손실: 2.18
에포크 1 
0번째 배치 손실: 1.90
100번째 배치 손실: 1.86
200번째 배치 손실: 1.81
300번째 배치 손실: 1.72
400번째 배치 손실: 1.80
에포크 2 
0번째 배치 손실: 1.58
100번째 배치 손실: 1.56
200번째 배치 손실: 1.49
300번째 배치 손실: 1.43
400번째 배치 손실: 1.49
에포크 3 
0번째 배치 손실: 1.32
100번째 배치 손실: 1.33
200번째 배치 손실: 1.23
300번째 배치 손실: 1.22
400번째 배치 손실: 1.25
에포크 4 
0번째 배치 손실: 1.13
100번째 배치 손실: 1.15
200번째 배치 손실: 1.04
300번째 배치 손실: 1.06
400번째 배치 손실: 1.09
에포크 5 
0번째 배치 손실: 0.99
100번째 배치 손실: 1.02
200번째 배치 손실: 0.90
300번째 배치 손실: 0.94
400번째 배치 손실: 0.97
에포크 6 
0번째 배치 손실: 0.89
100번째 배치 손실: 0.91
200번째 배치 손실: 0.80
300번째 배치 손실: 0.85
400번째 배치 손실: 0.89
에포크 7 
0번째 배치 손실: 0.81
100번째 배치 손실: 0.83
200번째 배치 손실: 0.72
300번째 배치 손실: 0.79
400번째 배치 손실: 0.82
에포크 8 
0번째 배치 손실: 0.74
100번째 배치 손실: 0.76
200번째 배치 손실: 0.66
300번째 배치 손실: 0.73
400번째 배치 손실: 0.77
에포크 9 
0번째 배치 손실: 0.69
100번째 배치 손실: 0.71
200번째 배치 손실: 0.62
300번째 배치 손실: 0.69
400번째 배치 손실: 0.73


In [26]:
predictions = model(test_images)
predictions = predictions.numpy()
predicted_labels = np.argmax(predictions, axis=1)
matches = predicted_labels == test_labels
print(f"정확도 {matches.mean():.2f}")

정확도 0.81


tf.Tensor(6.0, shape=(), dtype=float32)
